In [1]:
import pandas as pd

In [3]:
df_opp_stage_progress = pd.read_csv('/home/darshan/Documents/Vonage 2024-Nov-13/SFDC/Opportunities Stage Progression.csv')

df_opp_stage_progress.info()

/tmp/ipykernel_454802/1827992183.py:1: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  df_opp_stage_progress = pd.read_csv('/home/darshan/Documents/Vonage 2024-Nov-13/SFDC/Opportunities Stage Progression.csv')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 157580 entries, 0 to 157579
Data columns (total 16 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   Salesforce Instance     157580 non-null  object 
 1   Opportunity ID          157580 non-null  object 
 2   Opportunity Owner       157580 non-null  object 
 3   Opportunity Name        157573 non-null  object 
 4   Account Name            157562 non-null  object 
 5   Stage                   157580 non-null  object 
 6   Edited By               157580 non-null  object 
 7   Total ARR USD           157580 non-null  float64
 8   Field / Event           157580 non-null  object 
 9   Old Value               157580 non-null  object 
 10  New Value               157580 non-null  object 
 11  Edit Date               157580 non-null  object 
 12  Opportunity Geo-Region  157580 non-null  object 
 13  Type                    157580 non-null  object 
 14  Business Group      

In [54]:
df_opp_primary_contact = pd.read_csv('/home/darshan/Documents/Vonage 2024-Nov-13/SFDC/Opp with Primary Contact.csv', encoding='latin1')

df_opp_primary_contact.info()

/tmp/ipykernel_454802/2234851364.py:1: DtypeWarning: Columns (19,23,24) have mixed types. Specify dtype option on import or set low_memory=False.
  df_opp_primary_contact = pd.read_csv('/home/darshan/Documents/Vonage 2024-Nov-13/SFDC/Opp with Primary Contact.csv', encoding='latin1')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91181 entries, 0 to 91180
Data columns (total 35 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Salesforce Instance                91181 non-null  object 
 1   Opportunity ID                     91181 non-null  object 
 2   Owner Role                         87016 non-null  object 
 3   Opportunity Owner                  91181 non-null  object 
 4   Opportunity Name                   91175 non-null  object 
 5   Account ID                         91180 non-null  object 
 6   Account Name                       91171 non-null  object 
 7   Stage                              91181 non-null  object 
 8   Fiscal Period                      91181 non-null  object 
 9   Annual Recurring Revenue Currency  89173 non-null  object 
 10  Annual Recurring Revenue           91181 non-null  float64
 11  Amount Currency                    83781 non-null  obj

In [55]:
# Count the number of rows for each Opportunity ID in df_opp_stage_progress
opp_count = df_opp_stage_progress.groupby('Opportunity ID').size().reset_index(name='Row Count')

# Filter for Opportunity IDs with more than 1 row
multi_row_opp = opp_count[opp_count['Row Count'] > 1]

# Display the results
print(multi_row_opp)

        Opportunity ID  Row Count
8      0061400001F9Gte          2
16     0061400001H3TeM          2
24     0061400001I4nWZ          3
71     0061O00001Ls2Kd          2
83     0061O00001MdAfp          2
...                ...        ...
90594  006cg000003SW02          2
90596  006cg000003VcnG          3
90597  006cg000003XWVd          2
90598  006cg000003XnHx          2
90602  006cg000003d5MH          3

[49309 rows x 2 columns]


In [56]:
# Group by 'Opportunity ID' and 'Stage', count occurrences and get the latest 'Edit Date'
opp_stage_count = df_opp_stage_progress.groupby(['Opportunity ID', 'Stage']).agg(
    Stage_Progression_Count=('Opportunity ID', 'size'),
    Latest_Edit_Date=('Edit Date', 'max')
).reset_index()

# Display the result
print(opp_stage_count)

        Opportunity ID        Stage  Stage_Progression_Count  Latest_Edit_Date
0      00614000019lQ8S  Closed Lost                        1   6/30/2023 16:26
1      0061400001Cw8jI  Closed Lost                        1    3/4/2024 12:44
2      0061400001DQVhL  Closed Lost                        1    4/8/2024 15:45
3      0061400001DdU9q  Closed Lost                        1    4/8/2024 15:43
4      0061400001DemfE  Closed Lost                        1  11/10/2023 16:45
...                ...          ...                      ...               ...
90606  006cg000003hU4L    Discovery                        1    11/4/2024 9:41
90607  006cg000003hfB5     Proposal                        1  10/31/2024 10:28
90608  006cg000003jIn3      Meeting                        1   11/1/2024 10:34
90609  006cg000003mBkD      Meeting                        1    11/4/2024 5:52
90610  006cg000003mjYz      Meeting                        1    11/4/2024 9:54

[90611 rows x 4 columns]


In [57]:
# opp_stage_count.to_csv('opp_stage_count.csv', index=False)

In [58]:
# Check if any Opportunity ID is repeated
repeated_opp_ids = opp_stage_count[opp_stage_count.duplicated(subset='Opportunity ID', keep=False)]

# Display the repeated Opportunity IDs
print(repeated_opp_ids)

Empty DataFrame
Columns: [Opportunity ID, Stage, Stage_Progression_Count, Latest_Edit_Date]
Index: []


In [59]:
# Add 'SP_' prefix to columns of opp_stage_count
opp_stage_count.columns = ['SP_' + col if col != 'Opportunity ID' else col for col in opp_stage_count.columns]

# Perform a left join with df_opp_primary_contact on 'Opportunity ID'
merged_df = pd.merge(opp_stage_count, df_opp_primary_contact, on='Opportunity ID', how='left')

# Display the info of the resulting DataFrame
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90611 entries, 0 to 90610
Data columns (total 38 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Opportunity ID                     90611 non-null  object 
 1   SP_Stage                           90611 non-null  object 
 2   SP_Stage_Progression_Count         90611 non-null  int64  
 3   SP_Latest_Edit_Date                90611 non-null  object 
 4   Salesforce Instance                88060 non-null  object 
 5   Owner Role                         84395 non-null  object 
 6   Opportunity Owner                  88060 non-null  object 
 7   Opportunity Name                   88054 non-null  object 
 8   Account ID                         88059 non-null  object 
 9   Account Name                       88050 non-null  object 
 10  Stage                              88060 non-null  object 
 11  Fiscal Period                      88060 non-null  obj

In [60]:
# Create a new column 'stage_match' to check if 'SP_Stage' matches 'Stage'
merged_df['stage_match'] = merged_df['SP_Stage'] == merged_df['Stage']

# Display the info of the resulting DataFrame
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90611 entries, 0 to 90610
Data columns (total 39 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Opportunity ID                     90611 non-null  object 
 1   SP_Stage                           90611 non-null  object 
 2   SP_Stage_Progression_Count         90611 non-null  int64  
 3   SP_Latest_Edit_Date                90611 non-null  object 
 4   Salesforce Instance                88060 non-null  object 
 5   Owner Role                         84395 non-null  object 
 6   Opportunity Owner                  88060 non-null  object 
 7   Opportunity Name                   88054 non-null  object 
 8   Account ID                         88059 non-null  object 
 9   Account Name                       88050 non-null  object 
 10  Stage                              88060 non-null  object 
 11  Fiscal Period                      88060 non-null  obj

In [61]:
# Count how many stages match and how many do not
stage_match_count = merged_df['stage_match'].value_counts()

# Display the result
print(stage_match_count)

stage_match
True     87559
False     3052
Name: count, dtype: int64


In [62]:
# Count the number of empty values (null) in the 'SP_Stage' and 'Stage' columns
empty_stage_count = merged_df[['SP_Stage', 'Stage']].isnull().sum()

# Count how many stages match or do not match
stage_match_count = merged_df['stage_match'].value_counts()

# Create a summary report
report = {
    "Empty SP_Stage Count": empty_stage_count['SP_Stage'],
    "Empty Stage Count": empty_stage_count['Stage'],
    "Stage Match (True)": stage_match_count.get(True, 0),
    "Stage Match (False)": stage_match_count.get(False, 0)
}

# Display the report
print(report)

{'Empty SP_Stage Count': np.int64(0), 'Empty Stage Count': np.int64(2551), 'Stage Match (True)': np.int64(87559), 'Stage Match (False)': np.int64(3052)}


In [63]:
# Convert 'Created Date' to datetime
merged_df['Created Date'] = pd.to_datetime(merged_df['Created Date'], errors='coerce')

# Filter the data for the years 2023 and 2024
filtered_df = merged_df[merged_df['Created Date'].dt.year.isin([2023, 2024])]

# Display the info of the filtered DataFrame
filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 80212 entries, 3003 to 90610
Data columns (total 39 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   Opportunity ID                     80212 non-null  object        
 1   SP_Stage                           80212 non-null  object        
 2   SP_Stage_Progression_Count         80212 non-null  int64         
 3   SP_Latest_Edit_Date                80212 non-null  object        
 4   Salesforce Instance                80212 non-null  object        
 5   Owner Role                         78110 non-null  object        
 6   Opportunity Owner                  80212 non-null  object        
 7   Opportunity Name                   80207 non-null  object        
 8   Account ID                         80211 non-null  object        
 9   Account Name                       80204 non-null  object        
 10  Stage                              8

In [64]:
df_six_sense = pd.read_csv('/home/darshan/Documents/Vonage 2024-Nov-13/6Sense/6Sense.csv')

df_six_sense.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14229 entries, 0 to 14228
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   6sense Company Name    14114 non-null  object
 1   6sense Country         14114 non-null  object
 2   6sense Domain          14106 non-null  object
 3   6sense Revenue Range   14108 non-null  object
 4   6sense Employee Range  14110 non-null  object
 5   Industry               14108 non-null  object
 6   Industry (Legacy)      14105 non-null  object
 7   Stages                 14114 non-null  object
dtypes: object(8)
memory usage: 889.4+ KB


In [65]:
df_six_sense.sample()

,6sense Company Name,6sense Country,6sense Domain,6sense Revenue Range,6sense Employee Range,Industry,Industry (Legacy),Stages
910,1st Source Bank,United States,1stsource.com,$250M - $500M,"1,000 - 4,999",Financial Services,Financial Services,Decision


In [66]:
# Total count of records based on '6sense Company Name'
total_records = len(df_six_sense)

# Total count of null '6sense Company Name'
null_company_count = df_six_sense['6sense Company Name'].isnull().sum()

# Total unique '6sense Company Name'
unique_company_count = df_six_sense['6sense Company Name'].nunique()

# Display the results
print(f"Total Records: {total_records}")
print(f"Total Null 6sense Company Names: {null_company_count}")
print(f"Total Unique 6sense Company Names: {unique_company_count}")

Total Records: 14229
Total Null 6sense Company Names: 115
Total Unique 6sense Company Names: 14048


In [72]:
# Find the intersection of '6sense Company Name' and 'Account Name'
existing_company_count = df_six_sense['6sense Company Name'].dropna().isin(filtered_df['Account Name'].unique()).sum()

# Display the result
print(f"Company Names in 'filtered_df' Account Name: {existing_company_count}")

Company Names in 'filtered_df' Account Name: 680
